## BRONZE TO SILVER LAYER


In [4]:
import pandas as pd
from common_utilities import (
    global_path,
    replace_punctuation_from_columns,
)

### Bronze Layer - Trade History


In [5]:
# Initialize an empty list to store DataFrames
df_Ledger = []

# Loop through List of all CSV files in the folder
for file_path in global_path.ledger_bronze_layer_path.glob("*.csv"):
    # Read the CSV file
    df = pd.read_csv(file_path)
    # Append the DataFrame to the list
    df_Ledger.append(df)

# Concatenate all DataFrames into one
df_Ledger = pd.concat(df_Ledger, ignore_index=True)
df_Ledger = replace_punctuation_from_columns(df_Ledger)
df_Ledger.dropna(how="all", axis=1, inplace=True)

# # Convert 'bill_number' to int
# df_Ledger["bill_number"] = df_Ledger["bill_number"].fillna(0).astype(int)

# Add Datetime Col
df_Ledger["trade_date"] = pd.to_datetime(df_Ledger["trade_date"], format="%Y-%m-%d").dt.date
df_Ledger["settlement_date"] = pd.to_datetime(df_Ledger["settlement_date"], format="%Y-%m-%d").dt.date


# sort the dataframe by date
df_Ledger = df_Ledger.sort_values(by=["trade_date", "settlement_date", "exchange", "segment"])


df_Ledger = df_Ledger[
    [
        "wallet",
        "trade_date",
        "settlement_date",
        "exchange",
        "segment",
        "type",
        "narration",
        "debit",
        "credit",
        "closing_balance",
    ]
]

# Save the result as a csv file
df_Ledger.to_csv(global_path.ledger_silver_file_path, index=None)
df_Ledger.info()

print("SILVER Layer csv file for Bill Summary successfully created at:")
print(global_path.ledger_silver_file_path.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   wallet           100 non-null    object 
 1   trade_date       100 non-null    object 
 2   settlement_date  100 non-null    object 
 3   exchange         100 non-null    object 
 4   segment          100 non-null    object 
 5   type             100 non-null    object 
 6   narration        100 non-null    object 
 7   debit            59 non-null     float64
 8   credit           39 non-null     float64
 9   closing_balance  100 non-null    float64
dtypes: float64(3), object(7)
memory usage: 8.6+ KB
SILVER Layer csv file for Bill Summary successfully created at:
C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\Ledger\Ledger_data.csv


In [6]:
# for file_path in global_path.ledger_bronze_layer_path.glob("*.xlsx"):
#     # Read the xlsx file with header at line 8 (index 7 in pandas)
#     df = pd.read_excel(file_path, header=7)
#     df.to_csv(str(file_path).replace("Securities_846833.xlsx",".csv"), index=False)